**Dataset from: https://www.kaggle.com/datasets/alejopaullier/new-york-city-weather-data-2019**

## Data Pre-Processing
<br>
<i>idea</i>: Naive categorizing but use tavg to determine weather the day is Sunny or Cloudy, use precipitation in inches to determine if the day is rainy, and new_snow fall in inches to determine if the day is snowy. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

df = pd.read_csv('nyc_temperature.csv')
print(df.head())

# Remove unnecessary columns 
df = df.drop(columns=["tmax", "tmin", "departure", "HDD", "CDD", "snow_depth"])
df["precipitation"].replace(to_replace="T", value=0, inplace = True)
df["new_snow"].replace(to_replace="T", value=0, inplace = True)

#Data is already sorted by date 
print(df.head())


     date  tmax  tmin  tavg  departure  HDD  CDD precipitation new_snow  \
0  1/1/19    60    40  50.0       13.9   15    0          0.08        0   
1  2/1/19    41    35  38.0        2.1   27    0             0        0   
2  3/1/19    45    39  42.0        6.3   23    0             T        0   
3  4/1/19    47    37  42.0        6.5   23    0             0        0   
4  5/1/19    47    42  44.5        9.1   20    0          0.45        0   

  snow_depth  
0          0  
1          0  
2          0  
3          0  
4          0  
     date  tavg precipitation new_snow
0  1/1/19  50.0          0.08        0
1  2/1/19  38.0             0        0
2  3/1/19  42.0             0        0
3  4/1/19  42.0             0        0
4  5/1/19  44.5          0.45        0


## Prepare weather data 

In [15]:
def categorize_weather(row):
    if float(row['precipitation']) >= 0.1:
        return 'Rainy'
    elif float(row['new_snow']) > 0:
        return 'Snowy'
    elif float(row['tavg']) >= 60:
        return 'Sunny'
    else:
        return 'Cloudy'
df['weather'] = df.apply(categorize_weather, axis=1)
print(df[['date', 'tavg', 'precipitation', 'new_snow', 'weather']].head(100))

weather_data = list(df['weather'])
print(weather_data)

       date  tavg precipitation new_snow weather
0    1/1/19  50.0          0.08        0  Cloudy
1    2/1/19  38.0             0        0  Cloudy
2    3/1/19  42.0             0        0  Cloudy
3    4/1/19  42.0             0        0  Cloudy
4    5/1/19  44.5          0.45        0   Rainy
..      ...   ...           ...      ...     ...
95   6/4/19  52.5             0        0  Cloudy
96   7/4/19  55.0             0        0  Cloudy
97   8/4/19  63.0          0.25        0   Rainy
98   9/4/19  46.5          0.02        0  Cloudy
99  10/4/19  51.0             0        0  Cloudy

[100 rows x 5 columns]
['Cloudy', 'Cloudy', 'Cloudy', 'Cloudy', 'Rainy', 'Cloudy', 'Cloudy', 'Rainy', 'Cloudy', 'Cloudy', 'Cloudy', 'Cloudy', 'Cloudy', 'Cloudy', 'Cloudy', 'Cloudy', 'Cloudy', 'Snowy', 'Rainy', 'Rainy', 'Cloudy', 'Cloudy', 'Cloudy', 'Rainy', 'Cloudy', 'Cloudy', 'Cloudy', 'Cloudy', 'Rainy', 'Snowy', 'Cloudy', 'Cloudy', 'Cloudy', 'Cloudy', 'Cloudy', 'Cloudy', 'Rainy', 'Rainy', 'Rainy', 'Cloudy'

## Build and Normalize Transition Matrix

In [20]:
states = ['Sunny', 'Cloudy', 'Rainy', 'Snowy']
state_index = {state: i for i, state in enumerate(states)}

transition_matrix_counts = np.zeros((len(states), len(states)))

# Count transitions
for i in range(len(weather_data)-1):
    current_state = weather_data[i]
    next_state = weather_data[i+1]
    transition_matrix_counts[state_index[current_state]][state_index[next_state]] += 1

print("Transition Matrix Counts:")
print (transition_matrix_counts)

# Normalize the transition matrix 

transition_matrix = transition_matrix_counts.copy()
for i in range(len(states)):
    row_sum = np.sum(transition_matrix[i])
    if row_sum > 0:
        transition_matrix[i] = transition_matrix[i] / row_sum
        
print("Normalized Transition Matrix:")
print(transition_matrix)

Transition Matrix Counts:
[[102.   3.  25.   0.]
 [  4. 105.  31.   2.]
 [ 24.  31.  29.   3.]
 [  0.   3.   2.   0.]]
Normalized Transition Matrix:
[[0.78461538 0.02307692 0.19230769 0.        ]
 [0.02816901 0.73943662 0.21830986 0.01408451]
 [0.27586207 0.35632184 0.33333333 0.03448276]
 [0.         0.6        0.4        0.        ]]


## Predict Next N Day's Weather

In [ ]:
# Predict the weather for the next day
def predict_next_day(today_state):
    vector = np.zeros(4)
    vector[state_index[today_state]] = 1
    probabilities = vector @ transition_matrix
    return {state: round(prob, 4) for state, prob in zip(states, probabilities)}

print("\nPrediction for tomorrow if today is Sunny:")
print(predict_next_day("Sunny"))


# Predict the weather for the next N days
def predict_in_n_days(today_state, n):
    vector = np.zeros(4)
    vector[state_index[today_state]] = 1
    probabilities = vector @ np.linalg.matrix_power(transition_matrix, n)
    return {state: round(prob, 4) for state, prob in zip(states, probabilities)}

print("\nPrediction in 5 days if today is Sunny:")
print(predict_in_n_days("Sunny", 5))

# Simulate weather overtime
def simulate_weather(start_state, days):
    current = start_state
    forecast = [current]
    
    for _ in range(days):
        row = transition_matrix[state_index[current]]
        next_state = np.random.choice(states, p=row)
        forecast.append(next_state)
        current = next_state
        
    return forecast

print("\nSimulated 10-day forecast:")
print(simulate_weather("Sunny", 10))


Prediction for tomorrow if today is Sunny:
{'Sunny': 0.7846, 'Cloudy': 0.0231, 'Rainy': 0.1923, 'Snowy': 0.0}

Prediction in 5 days if today is Sunny:
{'Sunny': 0.4857, 'Cloudy': 0.2702, 'Rainy': 0.233, 'Snowy': 0.0112}

Simulated 10-day forecast:
['Sunny', 'Sunny', 'Sunny', 'Sunny', 'Rainy', 'Cloudy', 'Cloudy', 'Cloudy', 'Rainy', 'Sunny', 'Sunny']


**Graphs for First Order Markov**